# COMP 9130 - Mini Project 4: Deep Learning Classifier
## Fashion-MNIST Classification for StyleSort

**Framework:** PyTorch  
**Dataset:** Fashion-MNIST (60,000 train / 10,000 test, 28×28 grayscale, 10 classes)  
**Goal:** Build a product image classification system achieving >85% accuracy with business-aware error analysis.

### Business Context

StyleSort is an online fashion retailer processing over 100,000 product listings per month. Their current return rate is **32%**, significantly higher than the industry average of 20%. Customer surveys reveal that **40% of returns** happen because "the item wasn't what I expected" — often due to products being miscategorized in the catalog. Examples include formal Shirts listed as casual T-shirts, Ankle boots categorized as Sandals, and Coats listed as Pullovers.

### Our Approach

We train and compare **5 different neural network configurations** using PyTorch `nn.Module`, varying architecture depth, activation functions, hidden layer sizes, and batch normalization. We then perform business-oriented analysis including confusion matrix analysis, cost-weighted accuracy, confidence threshold analysis, and misclassification visualization.

---

## 1. Data Loading and Exploration

In [ ]:
from torchvision import datasets, transforms

transform = transforms.ToTensor()
train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

In [ ]:
import numpy as np

import matplotlib.pyplot as plt

# Class labels for FashionMNIST
class_names = train_data.classes

# Dataset statistics
print(f"Training set size: {len(train_data)}")
print(f"Test set size: {len(test_data)}")
print(f"Number of classes: {len(class_names)}")
print(f"Class names: {class_names}") 
print(f"Image shape: {train_data[0][0].shape}")

# Class distribution
train_labels = [label for _, label in train_data]
unique, counts = np.unique(train_labels, return_counts=True)
plt.figure(figsize=(12, 4))
plt.bar([class_names[i] for i in unique], counts)
plt.xlabel('Class')
plt.ylabel('Count')
plt.title('Training Data Class Distribution')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Display sample images
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for i, ax in enumerate(axes.flat):
    image, label = train_data[i]
    ax.imshow(image.squeeze(), cmap='gray')
    ax.set_title(class_names[label])
    ax.axis('off')
plt.tight_layout()
plt.show()

## 2. Data Preprocessing and Model Definition

We flatten the 28×28 images into 784-dimensional vectors and normalize pixel values to [0, 1]. A 90/10 train/validation split is used to monitor overfitting during training.

In [ ]:
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F

device = None
if torch.cuda.is_available():
    device = torch.device("cuda")
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():  # For Apple Silicon GPUs
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

# Flatten images and normalize to [0, 1]
x_train_flattened = torch.flatten(train_data.data, start_dim=1).float() / 255.0
y_train = train_data.targets

x_test_flattened = torch.flatten(test_data.data, start_dim=1).float() / 255.0
y_test = test_data.targets

train_data_flattened = torch.utils.data.TensorDataset(x_train_flattened, y_train)
test_data_flattened = torch.utils.data.TensorDataset(x_test_flattened, y_test)

loss_histories = []
accuracy_histories = []

print(f"Flattened training data shape: {x_train_flattened.shape}")
print(f"Flattened test data shape: {x_test_flattened.shape}")

test_loader = DataLoader(test_data_flattened, batch_size=64, shuffle=False)

# Create validation set (10% of training data)
val_size = int(0.1 * len(train_data_flattened))
train_size = len(train_data_flattened) - val_size
train_data_split, val_data = torch.utils.data.random_split(train_data_flattened, [train_size, val_size])
train_loader = DataLoader(train_data_split, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)

## 3. Model Definition (`nn.Module`)

We define a flexible `FashionClassifier` class using `nn.Module` that supports:
- Configurable number of hidden layers (1, 2, or 3)
- Configurable hidden layer size
- Configurable activation function (ReLU, LeakyReLU, ELU)
- Optional batch normalization
- Optional dropout

This allows us to systematically compare different architectures.

In [ ]:
class FashionClassifier(nn.Module):
    """
    Flexible MLP classifier for Fashion-MNIST.
    Supports configurable depth, width, activation, batch norm, and dropout.
    """
    def __init__(self, input_size=784, num_classes=10, hidden_layers=2,
                 activation_fn=nn.ReLU(), hidden_size=256,
                 batch_norm=False, dropout_prob=0.0):
        super().__init__()

        layers = []
        in_features = input_size

        for i in range(hidden_layers):
            out_features = hidden_size
            layers.append(nn.Linear(in_features, out_features))
            if batch_norm:
                layers.append(nn.BatchNorm1d(out_features))
            layers.append(activation_fn)
            if dropout_prob > 0:
                layers.append(nn.Dropout(dropout_prob))
            in_features = out_features

        layers.append(nn.Linear(in_features, num_classes))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)


# Quick test
test_model = FashionClassifier()
test_input = torch.randn(2, 784)
test_output = test_model(test_input)
print(f"Model output shape: {test_output.shape}")
print(f"Model parameters: {sum(p.numel() for p in test_model.parameters()):,}")

## 4. Custom Training Loop

We implement a manual training loop that:
1. Performs forward pass, computes cross-entropy loss, backward pass, and weight updates
2. Tracks training loss and validation accuracy per epoch
3. Returns loss and accuracy histories for plotting

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=5, device='cpu'):
    """
    Train the model and track loss/accuracy per epoch.
    
    Returns:
        loss_history: list of training losses per epoch
        accuracy_history: list of validation accuracies per epoch
    """
    loss_history = []
    accuracy_history = []

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        num_batches = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and weight update
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            num_batches += 1

        avg_loss = running_loss / num_batches
        loss_history.append(avg_loss)

        # Validation accuracy
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100.0 * correct / total
        accuracy_history.append(accuracy)

        print(f'Epoch [{epoch+1}/{epochs}] - Loss: {loss_history[-1]:.4f} - Val Acc: {accuracy:.2f}%')

    return loss_history, accuracy_history


def evaluate_model(model, test_loader, device='cpu'):
    """Evaluate model on test set and return accuracy."""
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100.0 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')
    return accuracy

## 5. Training 5 Model Configurations

We compare 5 different configurations to find the best architecture:

| Classifier | Layers | Activation | Hidden Size | Batch Norm | Dropout | Optimizer |
|-----------|--------|-----------|-------------|-----------|---------|----------|
| 1 (Best) | 3 | ELU | 512 | Yes | 0.0 | Adam |
| 2 | 3 | ReLU | 256 | No | 0.0 | Adam |
| 3 | 2 | ReLU | 256 | No | 0.0 | Adam |
| 4 | 3 | ReLU | 128 | Yes | 0.0 | Adam |
| 5 | 2 | ELU | 256 | Yes | 0.0 | AdamW |

These configurations were selected based on a prior hyperparameter grid search over hidden layers, activation functions, hidden sizes, optimizers, batch normalization, and dropout rates.

In [ ]:
classifiers = []
NUM_EPOCHS = 5

# ---- Classifier 1: 3 layers, ELU, 512, BN, Adam ----
print('=' * 60)
print('Classifier 1: 3 layers, ELU, hidden=512, BN=True, Adam')
print('=' * 60)
classifier1 = FashionClassifier(
    hidden_layers=3, activation_fn=nn.ELU(), hidden_size=512,
    batch_norm=True, dropout_prob=0.0
)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier1.parameters(), lr=0.001)
classifier1 = classifier1.to(device)
classifier1_loss_history, classifier1_accuracy_history = train_model(
    classifier1, train_loader, val_loader, criterion, optimizer, epochs=NUM_EPOCHS, device=device
)
classifier1_accuracy = evaluate_model(classifier1, test_loader, device=device)
loss_histories.append(classifier1_loss_history)
accuracy_histories.append(classifier1_accuracy_history)
classifiers.append(classifier1)
print()

# ---- Classifier 2: 3 layers, ReLU, 256, no BN, Adam ----
print('=' * 60)
print('Classifier 2: 3 layers, ReLU, hidden=256, BN=False, Adam')
print('=' * 60)
classifier2 = FashionClassifier(
    hidden_layers=3, activation_fn=nn.ReLU(), hidden_size=256,
    batch_norm=False, dropout_prob=0.0
)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier2.parameters(), lr=0.001)
classifier2 = classifier2.to(device)
classifier2_loss_history, classifier2_accuracy_history = train_model(
    classifier2, train_loader, val_loader, criterion, optimizer, epochs=NUM_EPOCHS, device=device
)
classifier2_accuracy = evaluate_model(classifier2, test_loader, device=device)
loss_histories.append(classifier2_loss_history)
accuracy_histories.append(classifier2_accuracy_history)
classifiers.append(classifier2)
print()

# ---- Classifier 3: 2 layers, ReLU, 256, no BN, Adam ----
print('=' * 60)
print('Classifier 3: 2 layers, ReLU, hidden=256, BN=False, Adam')
print('=' * 60)
classifier3 = FashionClassifier(
    hidden_layers=2, activation_fn=nn.ReLU(), hidden_size=256,
    batch_norm=False, dropout_prob=0.0
)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier3.parameters(), lr=0.001)
classifier3 = classifier3.to(device)
classifier3_loss_history, classifier3_accuracy_history = train_model(
    classifier3, train_loader, val_loader, criterion, optimizer, epochs=NUM_EPOCHS, device=device
)
classifier3_accuracy = evaluate_model(classifier3, test_loader, device=device)
loss_histories.append(classifier3_loss_history)
accuracy_histories.append(classifier3_accuracy_history)
classifiers.append(classifier3)
print()

# ---- Classifier 4: 3 layers, ReLU, 128, BN, Adam ----
print('=' * 60)
print('Classifier 4: 3 layers, ReLU, hidden=128, BN=True, Adam')
print('=' * 60)
classifier4 = FashionClassifier(
    hidden_layers=3, activation_fn=nn.ReLU(), hidden_size=128,
    batch_norm=True, dropout_prob=0.0
)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier4.parameters(), lr=0.001)
classifier4 = classifier4.to(device)
classifier4_loss_history, classifier4_accuracy_history = train_model(
    classifier4, train_loader, val_loader, criterion, optimizer, epochs=NUM_EPOCHS, device=device
)
classifier4_accuracy = evaluate_model(classifier4, test_loader, device=device)
loss_histories.append(classifier4_loss_history)
accuracy_histories.append(classifier4_accuracy_history)
classifiers.append(classifier4)
print()

# ---- Classifier 5: 2 layers, ELU, 256, BN, AdamW ----
print('=' * 60)
print('Classifier 5: 2 layers, ELU, hidden=256, BN=True, AdamW')
print('=' * 60)
classifier5 = FashionClassifier(
    hidden_layers=2, activation_fn=nn.ELU(), hidden_size=256,
    batch_norm=True, dropout_prob=0.0
)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(classifier5.parameters(), lr=0.001)
classifier5 = classifier5.to(device)
classifier5_loss_history, classifier5_accuracy_history = train_model(
    classifier5, train_loader, val_loader, criterion, optimizer, epochs=NUM_EPOCHS, device=device
)
classifier5_accuracy = evaluate_model(classifier5, test_loader, device=device)
loss_histories.append(classifier5_loss_history)
accuracy_histories.append(classifier5_accuracy_history)
classifiers.append(classifier5)

## 6. Training Curves

We plot the training loss and validation accuracy across epochs for all 5 classifiers to compare convergence speed and final performance.

In [ ]:
import os
os.makedirs('results', exist_ok=True)

# Plot loss histories
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
for i, loss_history in enumerate(loss_histories):
    plt.plot(range(1, len(loss_history) + 1), loss_history, marker='o', label=f'Classifier {i+1}')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.legend()
plt.grid(True)

# Plot accuracy histories
plt.subplot(1, 2, 2)
for i, accuracy_history in enumerate(accuracy_histories):
    plt.plot(range(1, len(accuracy_history) + 1), accuracy_history, marker='o', label=f'Classifier {i+1}')
plt.axhline(y=85, color='r', linestyle='--', alpha=0.5, label='85% Target')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Validation Accuracy Over Epochs')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig('results/training_curves.png', dpi=150, bbox_inches='tight')
plt.show()
print('Saved: results/training_curves.png')

In [ ]:
# Print experiment summary table
print('\n' + '=' * 65)
print(f'{"Classifier":<15} {"Test Accuracy":>15} {"Final Train Loss":>18}')
print('=' * 65)
test_accs = []
for i, clf in enumerate(classifiers):
    clf.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = clf(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = 100.0 * correct / total
    test_accs.append(acc)
    final_loss = loss_histories[i][-1]
    print(f'Classifier {i+1:<4} {acc:>14.2f}% {final_loss:>18.4f}')
print('=' * 65)

best_idx = np.argmax(test_accs)
print(f'\nBest model: Classifier {best_idx+1} with {test_accs[best_idx]:.2f}% test accuracy')
print(f'Target >85%: {"PASSED ✓" if test_accs[best_idx] > 85 else "NOT MET ✗"}')
best_classifier = classifiers[best_idx]

## 7. Confusion Matrix Analysis

The confusion matrix reveals which product categories are most frequently confused by the model. This is critical for StyleSort to understand where misclassification errors occur and to prioritize improvements in product descriptions and photography guidelines.

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns


for i, classifier in enumerate(classifiers):
    # Get predictions on test set
    classifier.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            outputs = classifier(images)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.numpy())

    # Create confusion matrix
    cm = confusion_matrix(all_labels, all_preds)

    # Plot confusion matrix
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'Confusion Matrix for Classifier {i+1}')
    plt.tight_layout()
    if i == best_idx:
        plt.savefig('results/confusion_matrix.png', dpi=150, bbox_inches='tight')
        print(f'Saved confusion matrix for best model (Classifier {i+1}): results/confusion_matrix.png')
    plt.show()

In [ ]:
# Analyze most confused category pairs for the best classifier
best_classifier.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = best_classifier(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.numpy())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

cm_best = confusion_matrix(all_labels, all_preds)

print('Most Confused Category Pairs (Best Model):')
print('=' * 55)

# Get off-diagonal elements
cm_no_diag = cm_best.copy()
np.fill_diagonal(cm_no_diag, 0)

# Find top 5 confused pairs
for _ in range(5):
    idx = np.unravel_index(cm_no_diag.argmax(), cm_no_diag.shape)
    count = cm_no_diag[idx]
    true_name = class_names[idx[0]]
    pred_name = class_names[idx[1]]
    print(f'  {true_name:>12} -> {pred_name:<12}: {count:>4} misclassifications')
    cm_no_diag[idx] = 0

print()
print('Key Insight: Shirt <-> T-shirt/top and Coat <-> Pullover are the most')
print('confused pairs. This aligns with the StyleSort business context — these')
print('categories have similar visual features in 28x28 grayscale but very')
print('different customer expectations (formal vs casual, heavy vs light).')

### Per-Class Performance Metrics

In [ ]:
# Per-class precision, recall, F1 for the best model
print('Per-Class Performance Metrics (Best Model):')
print('=' * 65)
report = classification_report(all_labels, all_preds, target_names=class_names, digits=4)
print(report)

## 8. Error Cost Analysis

Not all misclassifications are equally costly for StyleSort. We define a cost matrix based on business impact:

| Error Type | Business Impact | Cost Level |
|-----------|----------------|------------|
| Bag → Sneaker | Obvious to customer, quick return | Low (1) |
| Shirt → T-shirt | Customer expects different style | High (3) |
| Coat → Pullover | Customer expects warmth, different size | High (3) |
| Sandal → Sneaker | Wrong season, wrong use case | Medium (2) |
| Ankle boot → Sneaker | Similar use, price difference | Medium (2) |

In [ ]:
# Create a cost matrix that penalizes certain misclassifications more heavily
cost_matrix = np.array([
    [0,             1,          1,          1,      1,      1,      3,      1,          1,      1],      # T-shirt/top
    [1,             0,          1,          1,      1,      1,      1,      1,          1,      1],      # Trouser
    [1,             1,          0,          1,      3,      1,      1,      1,          1,      1],      # Pullover
    [1,             1,          1,          0,      1,      1,      1,      1,          1,      1],      # Dress
    [1,             1,          3,          1,      0,      1,      1,      1,          1,      1],      # Coat
    [1,             1,          1,          1,      1,      0,      1,      2,          1,      1],      # Sandal
    [3,             1,          1,          1,      1,      1,      0,      1,          1,      1],      # Shirt
    [1,             1,          1,          1,      1,      2,      1,      0,          0.5,    2],      # Sneaker
    [1,             1,          1,          1,      1,      1,      1,      0.5,        0,      1],      # Bag
    [1,             1,          1,          1,      1,      1,      1,      2,          1,      0],      # Ankle boot
    #t-shirt/top,   trouser,    pullover,   dress,  coat,   sandal, shirt,  sneaker,    bag,    ankle boot
])

# Visualize cost matrix
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(
    cost_matrix, annot=True, fmt='.1f', cmap='YlOrRd',
    xticklabels=class_names, yticklabels=class_names,
    ax=ax, linewidths=0.5
)
ax.set_xlabel('Predicted Label', fontsize=13)
ax.set_ylabel('True Label', fontsize=13)
ax.set_title('StyleSort Error Cost Matrix', fontsize=15)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# Calculate standard and cost-weighted accuracy for each classifier
for i, classifier in enumerate(classifiers):
    classifier.eval()
    c_preds = []
    c_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            outputs = classifier(images)
            _, predicted = torch.max(outputs, 1)
            c_preds.extend(predicted.cpu().numpy())
            c_labels.extend(labels.numpy())

    c_preds = np.array(c_preds)
    c_labels = np.array(c_labels)
    
    # Standard accuracy
    standard_accuracy = np.mean(c_preds == c_labels) * 100
    
    # Cost-weighted accuracy: penalize incorrect predictions based on cost matrix
    total_cost = 0
    for true_label, pred_label in zip(c_labels, c_preds):
        total_cost += cost_matrix[true_label, pred_label]
    
    cost_weighted_accuracy = (1 - (total_cost / (len(c_labels) * np.max(cost_matrix)))) * 100
    
    print(f"Classifier {i+1}:")
    print(f"  Standard Accuracy: {standard_accuracy:.2f}%")
    print(f"  Cost-Weighted Accuracy: {cost_weighted_accuracy:.2f}%")
    print(f"  Total Cost: {total_cost}")
    print()

## 9. Confidence Threshold Analysis

StyleSort needs to flag low-confidence predictions for human review before publishing. We analyze how different confidence thresholds affect:
- **Accuracy** on accepted (confident) predictions
- **Percentage** of items that would need human review

The optimal threshold balances high accuracy with minimal human review workload. For StyleSort's 10,000 daily new listings, this determines how many items require manual review.

In [ ]:
# Get softmax probabilities from the best classifier
best_classifier.eval()
all_preds_conf = []
all_labels_conf = []
all_probs = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = best_classifier(images)
        probs = F.softmax(outputs, dim=1)
        _, predicted = torch.max(outputs, 1)

        all_preds_conf.extend(predicted.cpu().numpy())
        all_labels_conf.extend(labels.numpy())
        all_probs.extend(probs.cpu().numpy())

all_preds_conf = np.array(all_preds_conf)
all_labels_conf = np.array(all_labels_conf)
all_probs = np.array(all_probs)

# Confidence threshold analysis
max_probs = np.max(all_probs, axis=1)
thresholds = np.arange(0.50, 1.00, 0.01)
accuracies_at_t = []
pct_accepted_at_t = []
pct_review_at_t = []

for t in thresholds:
    mask = max_probs >= t
    if mask.sum() == 0:
        accuracies_at_t.append(0.0)
        pct_accepted_at_t.append(0.0)
        pct_review_at_t.append(100.0)
    else:
        acc = (all_preds_conf[mask] == all_labels_conf[mask]).mean() * 100
        pct = mask.mean() * 100
        accuracies_at_t.append(acc)
        pct_accepted_at_t.append(pct)
        pct_review_at_t.append(100.0 - pct)

# Plot confidence threshold analysis
fig, ax1 = plt.subplots(figsize=(10, 6))

color1 = 'tab:blue'
ax1.set_xlabel('Confidence Threshold', fontsize=12)
ax1.set_ylabel('Accuracy on Accepted (%)', color=color1, fontsize=12)
ax1.plot(thresholds, accuracies_at_t, color=color1, linewidth=2, label='Accuracy')
ax1.tick_params(axis='y', labelcolor=color1)

ax2 = ax1.twinx()
color2 = 'tab:orange'
ax2.set_ylabel('Items Accepted (%)', color=color2, fontsize=12)
ax2.plot(thresholds, pct_accepted_at_t, color=color2, linewidth=2, label='% Accepted')
ax2.tick_params(axis='y', labelcolor=color2)

# Mark the 80% confidence threshold
ax1.axvline(x=0.80, color='gray', linestyle=':', alpha=0.6, label='80% threshold')

fig.suptitle('Confidence Threshold Analysis for StyleSort', fontsize=14)
fig.legend(loc='upper center', bbox_to_anchor=(0.5, 0.92), ncol=3, fontsize=11)
ax1.grid(True, alpha=0.3)
plt.tight_layout(rect=[0, 0, 1, 0.93])
plt.savefig('results/confidence_threshold.png', dpi=150, bbox_inches='tight')
plt.show()
print('Saved: results/confidence_threshold.png')

In [ ]:
# Print specific threshold analysis
print('Confidence Threshold Analysis Summary:')
print('=' * 65)
print(f'{"Threshold":>10} {"Accuracy":>12} {"Accepted":>12} {"Need Review":>12}')
print('-' * 65)
for t_val in [0.50, 0.60, 0.70, 0.80, 0.85, 0.90, 0.95, 0.99]:
    idx = int(round((t_val - 0.50) / 0.01))
    if idx < len(thresholds):
        print(f'{t_val:>10.2f} {accuracies_at_t[idx]:>11.2f}% '
              f'{pct_accepted_at_t[idx]:>11.2f}% {pct_review_at_t[idx]:>11.2f}%')
print('=' * 65)

# Specific analysis for 80% threshold
t80_idx = int(round((0.80 - 0.50) / 0.01))
print(f'\nAt 80% confidence threshold:')
print(f'  - {pct_review_at_t[t80_idx]:.1f}% of items would need human review')
print(f'  - Accuracy on accepted items: {accuracies_at_t[t80_idx]:.2f}%')
print(f'  - For 10,000 daily listings: ~{int(10000 * pct_review_at_t[t80_idx] / 100)} items need review')

## 10. Misclassified Examples

Visualizing misclassified images helps understand where the model struggles and can inform improvements to StyleSort's product photography guidelines.

In [ ]:
# Visualize 10 misclassified images with confidence scores
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.flatten()

# Get misclassified images from the best classifier
best_classifier.eval()
misclassified_images = []
misclassified_true_labels = []
misclassified_pred_labels = []
misclassified_confidences = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = best_classifier(images)
        probs = F.softmax(outputs, dim=1)
        max_conf, predicted = torch.max(probs, 1)
        
        # Find misclassified samples
        mask = predicted != labels
        if mask.sum() > 0:
            misclassified_images.extend(images[mask].cpu())
            misclassified_true_labels.extend(labels[mask].cpu().numpy())
            misclassified_pred_labels.extend(predicted[mask].cpu().numpy())
            misclassified_confidences.extend(max_conf[mask].cpu().numpy())
        
        # Stop once we have at least 10 misclassified samples
        if len(misclassified_images) >= 10:
            break

# Plot the first 10 misclassified images
for i in range(10):
    # Reshape back to 28x28 for visualization
    img = misclassified_images[i].view(28, 28)
    axes[i].imshow(img, cmap='gray')
    conf = misclassified_confidences[i] * 100
    axes[i].set_title(
        f'True: {class_names[misclassified_true_labels[i]]}\n'
        f'Pred: {class_names[misclassified_pred_labels[i]]}\n'
        f'Conf: {conf:.1f}%',
        fontsize=9, color='red'
    )
    axes[i].axis('off')

plt.tight_layout()
plt.suptitle('Misclassified Examples (Best Model)', fontsize=14, y=1.02)
plt.savefig('results/misclassified_examples.png', dpi=150, bbox_inches='tight')
plt.show()
print('Saved: results/misclassified_examples.png')

In [ ]:
# Analyze misclassification patterns
print('Misclassification Analysis:')
print('=' * 55)
print()
print('Why does the model make these mistakes?')
print('-' * 55)
print('1. Shirt <-> T-shirt/top: Both are upper-body garments.')
print('   In grayscale 28x28, the distinction between formal')
print('   (buttoned) shirts and casual T-shirts is subtle.')
print()
print('2. Coat <-> Pullover: Both are long-sleeved outerwear.')
print('   The difference in thickness/structure is hard to')
print('   capture in low-resolution grayscale images.')
print()
print('3. Sneaker <-> Ankle boot: Both are closed footwear.')
print('   The ankle height difference is the main distinguishing')
print('   feature, which can be ambiguous at 28x28 resolution.')
print()
print('4. Sandal <-> Sneaker: Generally well-separated, but')
print('   some sandal designs with straps can resemble sneakers.')

## 11. Business Recommendations for StyleSort

In [ ]:
# Compute best model test accuracy for recommendations
best_test_acc = test_accs[best_idx]
t80_review = pct_review_at_t[t80_idx]

print('=' * 65)
print('BUSINESS RECOMMENDATIONS FOR STYLESORT')
print('=' * 65)
print()
print('1. DEPLOYMENT STRATEGY')
print('-' * 40)
print(f'   - Deploy Classifier {best_idx+1} (test accuracy: {best_test_acc:.2f}%)')
print('   - Use 80% confidence threshold for auto-classification')
print(f'   - Expected ~{int(10000 * t80_review / 100)} of 10,000 daily items need human review')
print()
print('2. HIGH-PRIORITY IMPROVEMENTS')
print('-' * 40)
print('   - Shirt vs T-shirt: Add "collar type" and "button" tags')
print('     to product photography guidelines')
print('   - Coat vs Pullover: Require side-view photos showing')
print('     garment thickness and structure')
print()
print('3. HUMAN REVIEW WORKFLOW')
print('-' * 40)
print('   - Auto-accept predictions with >80% confidence')
print('   - Flag low-confidence items for human review')
print('   - Prioritize review of Shirt/T-shirt and Coat/Pullover')
print('     predictions (highest business cost if wrong)')
print()
print('4. COST REDUCTION')
print('-' * 40)
print('   - Focus on reducing high-cost errors (Shirt<->T-shirt,')
print('     Coat<->Pullover) rather than all errors equally')
print('   - Consider class-weighted loss function for fine-tuning')
print()
print('5. FUTURE IMPROVEMENTS')
print('-' * 40)
print('   - Use CNN architecture for better spatial feature extraction')
print('   - Collect higher-resolution color product images')
print('   - Implement data augmentation for underperforming classes')
print('   - Consider transfer learning from pre-trained models')
print('   - Increase training epochs with early stopping for')
print('     better convergence without overfitting')

## 12. Summary

### Key Results
- **5 model configurations** were compared, varying depth (2–3 layers), activation (ReLU, ELU), hidden size (128–512), batch normalization, and optimizer (Adam, AdamW)
- **Best model (Classifier 1):** 3 hidden layers, ELU activation, 512 hidden units, batch normalization, Adam optimizer — achieved the highest test accuracy
- All models exceeded the **85% accuracy target** required by StyleSort

### What Worked Best
- **Batch normalization** significantly improved training stability and final accuracy
- **ELU activation** provided a slight edge over ReLU in deeper architectures
- **Larger hidden sizes (512)** outperformed smaller ones (128, 256) when combined with batch normalization

### Business Impact
- The model can automatically categorize the majority of StyleSort's 10,000 daily listings
- Confidence-based flagging enables efficient human-in-the-loop review
- The most problematic category pairs (Shirt↔T-shirt, Coat↔Pullover) are identified for targeted improvement
- Focused improvements on high-cost error pairs can further reduce the 32% return rate